In [2]:
from transformers import MarianMTModel , MarianTokenizer
import pandas as pd

In [ ]:
data = pd.read_csv('./data/splits/text/train_lang.tsv',sep='\t')
data

In [ ]:
def translate(source_text,src_lang,tgt_lang):
    mname = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'
    model = MarianMTModel.from_pretrained(mname)
    tok = MarianTokenizer.from_pretrained(mname)
    
    batch_enc = tok.prepare_seq2seq_batch(source_text, return_tensors="pt")
    
    gen = model.generate(**batch_enc)
    
    words = tok.batch_decode(gen, skip_special_tokens=True)
    
    return words

In [ ]:
sample_text = ["show me the flights from st. petersburg to salt lake city that arrive before noon","is there ground transportation from the pittsburgh airport to downtown pittsburgh"]

In [ ]:
aug_data = []

In [ ]:
# translate english to french
EN_data = list(data[data.Language == 'en']['utterance'])
FR_data = list(data[data.Language == 'fr']['utterance'])

In [ ]:
# direct augmentation
EN_translated_ENFR = translate(EN_data,'en','fr')
FR_translated_FREN = translate(EN_data,'fr','en')

In [ ]:
# self augmentation
Aug_ENDE = translate(EN_data,'en','de')
EN_translated_DEEN = translate(Aug_ENFR,'de','en')

Aug_FRDE = translate(FR_data,'fr','de')
FR_translated_DEFR = translate(Aug_FRDE,'de','fr')

In [ ]:
# one hop cross translation
EN_DE_translation = translate(EN_data,'en','de')
DE_FR_translation = translate(EN_DE_translation,'de','fr')

FR_DE_translation = translate(FR_data,'fr','de')
DE_EN_translation = translate(FR_DE_translation,'de','en')

In [1]:
aug_data += EN_data
aug_data += FR_data
aug_data += EN_translated_ENFR
aug_data += FR_translated_FREN
aug_data += EN_translated_DEEN
aug_data += FR_translated_DEFR
aug_data += DE_FR_translation
aug_data += DE_EN_translation

NameError: name 'aug_data' is not defined

In [4]:
augDataPD = pd.DataFrame({'utterance':aug_data})

NameError: name 'aug_data' is not defined

In [ ]:
augDataPD.to_csv('./data/splits/text/augment_train.txt')